In [1]:
import pandas, numpy

## functions & options


In [2]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7//0 in_silico/Python/1)data_input/'
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

In [5]:
#define a function to split after the ".". The 1 is for how many word to have. if we have several dot. The [1] is what is after the dot, and [0] what is before
def split(L):
  return L.split(".",1)[0]

# II] Dataframe settings

In [6]:
%%time
path = input_file_directory + "protein_coding_list.txt"
protein_coding_list= pandas.read_csv(path, sep = "\t")
print(protein_coding_list.shape)
protein_coding_list.head()

(22796, 4)
Wall time: 44.9 ms


,ensembl_gene_id,hgnc_symbol,entrezgene_id,transcript_biotype
1,ENSG00000198888,MT-ND1,4535.0,protein_coding
2,ENSG00000198763,MT-ND2,4536.0,protein_coding
3,ENSG00000198804,MT-CO1,4512.0,protein_coding
4,ENSG00000198712,MT-CO2,4513.0,protein_coding
5,ENSG00000228253,MT-ATP8,4509.0,protein_coding


In [7]:
%%time
#Download the sample labels for normal and primary
path = input_file_directory + "/samples_IDs/samples_normal_tissue.tsv"
normal_tissue_IDs = pandas.read_csv(path, sep = "\t")

path = input_file_directory + "/samples_IDs/samples_primary_tumors.tsv"
primary_tumor_IDs = pandas.read_csv(path, sep = "\t")

#Convert the labels to list
normal_samples_labels = normal_tissue_IDs['sample'].to_list()
primary_samples_labels = primary_tumor_IDs['sample'].to_list()


Wall time: 46.9 ms


In [8]:
%%time
#Download the big expression data
path = input_file_directory + "/xenabrowser_brut_data/TcgaTargetGtex_rsem_gene_tpm.tsv"
df_gexpr_all = pandas.read_csv(path, sep = "\t", index_col='sample')

print(df_gexpr_all.shape)
df_gexpr_all.head(2)

(60498, 19131)
Wall time: 14min 39s


,GTEX-S4Q7-0003-SM-3NM8M,TCGA-19-1787-01,TCGA-S9-A7J2-01,GTEX-QV31-1626-SM-2S1QC,TCGA-G3-A3CH-11,TCGA-B5-A5OE-01,GTEX-13QIC-0011-R1a-SM-5O9CJ,TCGA-B2-5641-11,GTEX-ZPCL-0126-SM-4WWC8,TARGET-20-PANGDN-09,...,TCGA-FI-A2EY-01,TCGA-55-6985-11,TCGA-EJ-5527-01,TCGA-G3-A25X-01,TCGA-24-2254-01,GTEX-11ZTS-3326-SM-5LU9Y,GTEX-VJYA-0726-SM-4KL1T,GTEX-ZA64-2126-SM-5Q5A8,GTEX-Q2AG-2826-SM-2HMJQ,GTEX-XV7Q-0426-SM-4BRVN
sample,,,,,,,,,,,,,,,,,,,,,
ENSG00000242268.2,-3.4580,-9.9658,0.2998,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-4.0350,-2.0529,-9.9658,-9.9658,-1.9379,1.5165,-9.9658,-2.3884,0.0440,-3.3076
ENSG00000259041.1,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,...,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658,-9.9658


In [33]:
list_atg8 = protein_coding_list.loc[protein_coding_list['hgnc_symbol'].isin(['MAP1LC3A', 'MAP1LC3B','MAP1LC3B2',
                                                                             'MAP1LC3C','GABARAP',
                                                                             'GABARAPL1', 'GABARAPL2'])]
list_atg8_ens = list_atg8['ensembl_gene_id'].to_list()

In [17]:
%%time
#remove dot in ENSEMBL name. Function split created at the beginning
#remove dot in ens name
df_gexpr_all = df_gexpr_all.reset_index()
df_gexpr_all["sample"] = df_gexpr_all["sample"].apply(split)
#see if have duplicate
print('number of duplicated genes:', df_gexpr_all.duplicated('sample').sum())

#Merge with prot coding list
df_gexpr_all.rename(columns = {'sample' : 'ensembl_gene_id'}, inplace = True)
dfmerged = protein_coding_list.merge(df_gexpr_all)
print(dfmerged.shape)
dfmerged.head(2)

number of duplicated genes: 0
(19536, 19135)
Wall time: 1min 14s


,ensembl_gene_id,hgnc_symbol,entrezgene_id,transcript_biotype,GTEX-S4Q7-0003-SM-3NM8M,TCGA-19-1787-01,TCGA-S9-A7J2-01,GTEX-QV31-1626-SM-2S1QC,TCGA-G3-A3CH-11,TCGA-B5-A5OE-01,...,TCGA-FI-A2EY-01,TCGA-55-6985-11,TCGA-EJ-5527-01,TCGA-G3-A25X-01,TCGA-24-2254-01,GTEX-11ZTS-3326-SM-5LU9Y,GTEX-VJYA-0726-SM-4KL1T,GTEX-ZA64-2126-SM-5Q5A8,GTEX-Q2AG-2826-SM-2HMJQ,GTEX-XV7Q-0426-SM-4BRVN
0,ENSG00000198888,MT-ND1,4535.0,protein_coding,11.9390,13.1286,12.9213,12.3347,13.3366,10.0615,...,13.9398,12.9477,11.8238,9.8547,13.2571,13.0066,12.2818,13.3569,14.3486,11.6654
1,ENSG00000198763,MT-ND2,4536.0,protein_coding,11.7935,13.0793,12.7126,12.9118,13.4186,9.8850,...,14.3906,13.0265,11.7369,9.3562,13.0466,13.5249,12.6554,13.9404,14.7075,12.8655


In [18]:
#Control of the merge. See value of ENSG00000198712 before merging
print(df_gexpr_all.loc[df_gexpr_all['ensembl_gene_id'] == 'ENSG00000198712']['GTEX-QV31-1626-SM-2S1QC'])

#Control of the merge. See value of ENSG00000198712 after merging. have to be the same
print(dfmerged.loc[dfmerged['ensembl_gene_id'] == 'ENSG00000198712']['GTEX-QV31-1626-SM-2S1QC'])

28551    13.4472
Name: GTEX-QV31-1626-SM-2S1QC, dtype: float64
3    13.4472
Name: GTEX-QV31-1626-SM-2S1QC, dtype: float64


In [48]:
dfmerged = dfmerged.loc[dfmerged['ensembl_gene_id'].isin(list_atg8_ens)]
dfmerged.reset_index(drop=True, inplace=True)
dfmerged

,ensembl_gene_id,hgnc_symbol,entrezgene_id,transcript_biotype,GTEX-S4Q7-0003-SM-3NM8M,TCGA-19-1787-01,TCGA-S9-A7J2-01,GTEX-QV31-1626-SM-2S1QC,TCGA-G3-A3CH-11,TCGA-B5-A5OE-01,...,TCGA-FI-A2EY-01,TCGA-55-6985-11,TCGA-EJ-5527-01,TCGA-G3-A25X-01,TCGA-24-2254-01,GTEX-11ZTS-3326-SM-5LU9Y,GTEX-VJYA-0726-SM-4KL1T,GTEX-ZA64-2126-SM-5Q5A8,GTEX-Q2AG-2826-SM-2HMJQ,GTEX-XV7Q-0426-SM-4BRVN
0,ENSG00000101460,MAP1LC3A,84557.0,protein_coding,0.1903,5.4754,6.2561,5.4787,5.8069,6.0940,...,5.8230,4.9640,3.6916,4.9472,5.5414,7.1182,4.6294,6.7419,7.6294,5.4855
1,ENSG00000139112,GABARAPL1,23710.0,protein_coding,2.8502,5.0905,6.4440,7.6350,6.2202,6.1336,...,3.9543,6.6181,6.0622,4.0427,5.1420,7.7662,4.0833,7.7253,7.7078,6.3252
2,ENSG00000140941,MAP1LC3B,81631.0,protein_coding,5.8588,6.2487,6.2662,5.7811,3.6612,4.4562,...,1.9859,5.8617,5.3306,3.8905,5.6071,5.3367,6.1699,6.3911,6.9382,5.9461
3,ENSG00000258102,MAP1LC3B2,643246.0,protein_coding,2.2268,1.9340,3.4648,1.1050,0.0014,0.7832,...,-0.6643,1.0007,1.5998,0.0014,1.3454,-0.1187,1.5216,3.0671,4.3133,0.6790
4,ENSG00000034713,GABARAPL2,11345.0,protein_coding,7.1908,7.3861,8.2165,6.7594,5.3502,6.1697,...,3.7678,7.0101,7.2579,5.9486,7.6922,7.3957,6.0065,7.4558,7.9891,7.1368
5,ENSG00000170296,GABARAP,11337.0,protein_coding,8.6960,9.8680,10.0536,9.3901,8.3785,9.1940,...,7.8179,9.7045,9.4443,9.1028,9.8043,9.8057,8.2410,9.4135,9.0366,9.4867
6,ENSG00000197769,MAP1LC3C,440738.0,protein_coding,-9.9658,1.5612,-5.0116,-1.3183,-3.6259,1.0433,...,-5.5735,2.0604,-5.5735,-9.9658,-2.4659,1.5013,-3.3076,5.1611,-9.9658,-1.4699


In [49]:
%%time
#Isolate Normal and Primary tissues
##keep the gene_ids
df_part1 = dfmerged.iloc[:,0:4]

##subset
normal_protcoding = dfmerged.iloc[:,4:][normal_samples_labels]
primary_protcoding = dfmerged.iloc[:,4:][primary_samples_labels]

normal_protcoding = pandas.concat([df_part1, normal_protcoding], axis=1)
primary_protcoding = pandas.concat([df_part1, primary_protcoding], axis=1)

print(normal_protcoding.shape)
print(primary_protcoding.shape)
normal_protcoding

(7, 7433)
(7, 9189)
Wall time: 23.9 ms


,ensembl_gene_id,hgnc_symbol,entrezgene_id,transcript_biotype,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,...,GTEX-ZZPU-0726-SM-5N9C8,GTEX-ZZPU-0926-SM-5GZYT,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
0,ENSG00000101460,MAP1LC3A,84557.0,protein_coding,6.3886,6.0248,6.9068,7.2501,5.7986,6.8147,...,4.1136,6.1041,5.0122,5.8851,4.4270,5.9630,5.1473,5.1974,4.9584,5.6565
1,ENSG00000139112,GABARAPL1,23710.0,protein_coding,7.5650,10.1744,7.4675,7.3984,7.3886,7.9972,...,5.3851,7.2012,6.0771,6.4684,6.1687,6.8124,6.4433,4.4536,6.7240,7.2816
2,ENSG00000140941,MAP1LC3B,81631.0,protein_coding,6.3560,7.5498,6.4288,6.0758,5.3424,6.7644,...,3.7421,5.5976,5.7192,5.4896,4.2289,5.5027,5.0444,5.0000,6.3342,6.6532
3,ENSG00000258102,MAP1LC3B2,643246.0,protein_coding,1.6187,1.9034,1.5064,1.0293,0.7916,2.2421,...,-3.1714,0.3685,-0.2159,1.0222,-1.3183,-0.0277,3.3163,0.7748,1.1706,1.1184
4,ENSG00000034713,GABARAPL2,11345.0,protein_coding,6.8199,7.2244,6.4175,7.4548,6.6418,6.8382,...,5.1981,6.9927,7.2868,7.0491,5.9284,6.5670,6.7217,6.3409,6.8468,6.7245
5,ENSG00000170296,GABARAP,11337.0,protein_coding,9.7476,9.5465,9.6344,9.3360,8.6253,8.8726,...,8.5168,8.3210,9.7385,9.0121,8.8648,9.0458,9.3274,8.7760,8.7760,9.5656
6,ENSG00000197769,MAP1LC3C,440738.0,protein_coding,3.5498,2.4198,1.2756,2.7826,-2.7274,2.9929,...,-4.0350,-5.0116,-1.9942,-0.9686,0.7832,-0.7588,0.2154,-4.2934,-0.2498,1.6327


In [51]:
#test if everything is still good after concat and subset
print(dfmerged['TCGA-ZS-A9CG-01'][3])
print(primary_protcoding['TCGA-ZS-A9CG-01'][3])

0.2881
0.2881


In [52]:
print("Normal tissue")
print("is nan in ensembl:", normal_protcoding['ensembl_gene_id'].isna().sum())
print(' number of duplicate:',normal_protcoding.duplicated('ensembl_gene_id').sum())
print("is nan in symbol:",normal_protcoding['hgnc_symbol'].isna().sum())
print(' number of duplicate:',normal_protcoding.duplicated('hgnc_symbol').sum())
print()
print("Primary Tumor")
print("is nan in ensembl:", primary_protcoding['ensembl_gene_id'].isna().sum())
print(' number of duplicate:',primary_protcoding.duplicated('ensembl_gene_id').sum())
print("is nan in symbol:",primary_protcoding['hgnc_symbol'].isna().sum())
print(' number of duplicate:',primary_protcoding.duplicated('hgnc_symbol').sum())


Normal tissue
is nan in ensembl: 0
 number of duplicate: 0
is nan in symbol: 0
 number of duplicate: 0

Primary Tumor
is nan in ensembl: 0
 number of duplicate: 0
is nan in symbol: 0
 number of duplicate: 0


In [53]:
#see how many duplicates in genes
print('normal shape:', normal_protcoding.shape)
print('number of duplicate:',normal_protcoding.duplicated('ensembl_gene_id').sum())
print()
print('primary shape is:', primary_protcoding.shape)
print('number of duplicate:',primary_protcoding.duplicated('ensembl_gene_id').sum())
print()

#delete duplicates
normal_protcoding.drop_duplicates(subset = "ensembl_gene_id", inplace = True)
primary_protcoding.drop_duplicates(subset = "ensembl_gene_id", inplace = True)

normal shape: (7, 7433)
number of duplicate: 0

primary shape is: (7, 9189)
number of duplicate: 0



## A) ENSEMBL

In [55]:
%%time
#Preparation of the global dataframe
protcoding_final = []
data = [normal_protcoding, primary_protcoding]

for df in data:
    #transpose the table 
    df = numpy.transpose(df)

    #Change head column & delete the name of the column index; choose ENSEMBL
    df.columns = df.iloc[0]
    df.columns.name = None
    df = df.reset_index()

    #Delete the first rows & columns rename to merge 
    df = df.drop(df.index[0:4])
    df.rename(columns = {'index' : 'sample'}, inplace = True)

    protcoding_final.append(df)
    
ensembl_normal_protcoding_expr = protcoding_final[0]
ensembl_primary_protcoding_expr = protcoding_final[1]

print('normal shape =', ensembl_normal_protcoding_expr.shape)
print('primary shape =', ensembl_primary_protcoding_expr.shape)
ensembl_normal_protcoding_expr.head()

normal shape = (7429, 8)
primary shape = (9185, 8)
Wall time: 27.9 ms


,sample,ENSG00000101460,ENSG00000139112,ENSG00000140941,ENSG00000258102,ENSG00000034713,ENSG00000170296,ENSG00000197769
4,GTEX-1117F-0226-SM-5GZZ7,6.3886,7.565,6.356,1.6187,6.8199,9.7476,3.5498
5,GTEX-1117F-0426-SM-5EGHI,6.0248,10.1744,7.5498,1.9034,7.2244,9.5465,2.4198
6,GTEX-1117F-0526-SM-5EGHJ,6.9068,7.4675,6.4288,1.5064,6.4175,9.6344,1.2756
7,GTEX-1117F-0626-SM-5N9CS,7.2501,7.3984,6.0758,1.0293,7.4548,9.336,2.7826
8,GTEX-1117F-0726-SM-5GIEN,5.7986,7.3886,5.3424,0.7916,6.6418,8.6253,-2.7274


## B) SYMBOLS

In [56]:
%%time
#Preparation of the global dataframe
protcoding_final = []
data = [normal_protcoding, primary_protcoding]

for df in data:
    df = df.dropna(subset=['hgnc_symbol'])
    #transpose the table 
    df = numpy.transpose(df)

    #Change head column & delete the name of the column index; choose ENSEMBL
    df.columns = df.iloc[1]
    df.columns.name = None
    df = df.reset_index()

    #Delete the first rows & columns rename to merge 
    df = df.drop(df.index[0:4])
    df.rename(columns = {'index' : 'sample'}, inplace = True)

    protcoding_final.append(df)
    
symbol_normal_protcoding = protcoding_final[0]
symbol_primary_protcoding = protcoding_final[1]

print('normal shape =', symbol_normal_protcoding.shape)
print('primary shape =', symbol_primary_protcoding.shape)
symbol_normal_protcoding.head()

normal shape = (7429, 8)
primary shape = (9185, 8)
Wall time: 76.8 ms


,sample,MAP1LC3A,GABARAPL1,MAP1LC3B,MAP1LC3B2,GABARAPL2,GABARAP,MAP1LC3C
4,GTEX-1117F-0226-SM-5GZZ7,6.3886,7.565,6.356,1.6187,6.8199,9.7476,3.5498
5,GTEX-1117F-0426-SM-5EGHI,6.0248,10.1744,7.5498,1.9034,7.2244,9.5465,2.4198
6,GTEX-1117F-0526-SM-5EGHJ,6.9068,7.4675,6.4288,1.5064,6.4175,9.6344,1.2756
7,GTEX-1117F-0626-SM-5N9CS,7.2501,7.3984,6.0758,1.0293,7.4548,9.336,2.7826
8,GTEX-1117F-0726-SM-5GIEN,5.7986,7.3886,5.3424,0.7916,6.6418,8.6253,-2.7274


In [60]:
dfs = symbol_normal_protcoding, symbol_primary_protcoding
info = "normal", "primary"

for df in dfs:
    df.set_index("sample", inplace=True)
    df.loc['mean'] = df.mean()
    df.loc['median'] = df.median()

In [63]:
Normal Tissues

,MAP1LC3A,GABARAPL1,MAP1LC3B,MAP1LC3B2,GABARAPL2,GABARAP,MAP1LC3C
sample,,,,,,,
mean,5.95677,6.5104,5.60421,0.752485,6.8238,8.98835,-1.33797
median,6.2284,6.69775,5.7384,0.8164,6.93575,9.14835,-1.0262
